## Case Study (Predict Bankruptcy)

In [9]:
# installing packages

import pandas as pd
import numpy as np

# for visualisation
import plotly.io as pio
pio.templates.default = "plotly_dark"
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msn
import matplotlib.pyplot as plt

from typing import Tuple, Optional


In [2]:
# setting display constraints
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [3]:
def read_xlsx(path: str, sheet_name: str) -> pd.DataFrame:
    """
    This function read the XLSX files given a sheetname 
    """
    try:
        data = pd.read_excel(open(path,"rb"), sheet_name=sheet_name)
    except FileNotFoundError:
        print("file not found")
    return data


In [4]:
path = "../data/business_bankruptcy_prediction.xlsx"
data_sheet = "data"

data = read_xlsx(path=path, sheet_name=data_sheet)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 97 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 1    ROA(A) before interest and % after tax                   6819 non-null   float64
 2    ROA(B) before interest and depreciation after tax        6818 non-null   float64
 3    Operating Gross Margin                                   6819 non-null   float64
 4    Realized Sales Gross Margin                              6818 non-null   object 
 5    Operating Profit Rate                                    6819 non-null   float64
 6    Pre-tax net Interest Rate                                6819 non-null   float64
 7    After-tax net Interest Rate                              6819 non-null   float64
 8    Non-industry inco

In [6]:

def check_missing(data: pd.DataFrame, plot:bool=False)-> str:
     """
     The function checks if the data has missing features and displays them
     """
     
     if plot:
          msn.matrix(data, figsize=(45,8),sparkline=False)
     missing_total = data.isna().sum().sum()
     print(f"There are in total {missing_total} missing values in the data.")
     if missing_total !=0:
          print(f"Following are the list of columns with their corresponding missing number of values.\
          \n{data.isna().sum()[data.isna().sum()>0]}")

check_missing(data=data)

There are in total 8 missing values in the data.
Following are the list of columns with their corresponding missing number of values.          
 ROA(B) before interest and depreciation after tax    1
 Realized Sales Gross Margin                          1
 Net Value Per Share (C)                              1
 Per Share Net profit before tax (Yuan ¬•)            1
 Regular Net Profit Growth Rate                       1
 Net Value Growth Rate                                1
 Inventory and accounts receivable/Net value          1
 Inventory/Working Capital                            1
dtype: int64


In [7]:
data_nomsn = data.dropna()
check_missing(data=data_nomsn)

There are in total 0 missing values in the data.


In [225]:
def plot_bar(data1: pd.Series, data2: pd.Series, title:str, 
index1:Optional[list]=None, index2:Optional[list]=None,custom_index:bool=False) -> go.Figure:
    """
    Uses plotly to plot bar charts with a simplified interface
    """
    fig=make_subplots(1,2)
    
    if custom_index:
        fig.append_trace(go.Bar(x=index1, y=data1.values,
        name="values"), row=1, col=1)

        fig.append_trace(go.Bar(x=index2, 
                    y=np.round(data2.values,2)*100,
                    name="%"),
                    row=1, col=2)
   
    else:
        fig.append_trace(go.Bar(x=data1.index, y=data1.values,
        name="values"), row=1, col=1)

        fig.append_trace(go.Bar(x=data2.index, 
                    y=np.round(data2.values,2)*100,
                    name="%"),
                    row=1, col=2)

    fig.update_layout(title=title,width=1000, height=500,
    yaxis_title="frequency",yaxis2_title="%",
    xaxis_tickangle=-90,xaxis2_tickangle=-90)
    
    fig.show()

def get_dist(data:pd.DataFrame, col:str)-> Tuple[pd.DataFrame, pd.DataFrame]:
    perc = data[col].value_counts(normalize=True)
    num = data[col].value_counts()
    return (num, perc)

target_dist_num, target_dist_perc = get_dist(data=data_nomsn,col="Bankruptcy")
target_index = ["Thrive","Bankruptcy"]
plot_bar(target_dist_num,target_dist_perc,title="Data Distribution", custom_index=True, 
index1=target_index,index2=target_index)

In [226]:
data_nomsn["Company type"].value_counts()

company_dist_num, company_dist_perc = get_dist(data=data_nomsn,col="Company type")
plot_bar(company_dist_num,company_dist_perc,title="Data Distribution")

In [227]:
grp1=["Company type","Bankruptcy"]

data_agg = (
    data_nomsn.groupby(grp1)
    .agg({"Bankruptcy":"count"})
    )
data_agg["Bankruptcy %"] = (data_agg
.groupby(level=0)
.apply(lambda x:round(x / float(x.sum()),2)))

In [228]:
data_agg = data_agg.rename(columns={"Bankruptcy": "values", "Bankruptcy %":"perc%"}).reset_index()
data_agg["index"] = data_agg["Company type"].astype(str)+"_"+data_agg["Bankruptcy"].astype(str)

data_agg.set_index("index", inplace = True)
data_agg = data_agg.drop(grp1, axis=1)

In [229]:
comp_type_index = list(data_agg.index.values)
plot_bar(data_agg["values"],data_agg["perc%"], title="Bankruptcy")